# 🏷️ Part 3.3 - Classify Activities as AI-Augmentable or Not

**Author:** Yu Kyung Koh  
**Last Updated:** August 1, 2025  

---

### 🎯 Objective
* Use GPT to label whether each standardized activity is AI-exposed. Specifically, GPT is asked to assign two scores to each task:

    *  **AI Substitutability** - Can AI fully or partially replace the human?
    *  **AI Complementarity** - Can AI enhance or support human performance?

* Each score is rated on a **three-point scale**: 0, 0.5, or 1

### 🗂️ Outline
* **Section 1:** Load the standardized job activities data
* **Section 2:** Use GPT to get AI exposure scores for each standardized task

---
## Section 1: Load the standardized job activities data

In [3]:
import pandas as pd
import os
import re
import joblib
from tqdm import tqdm
from joblib import Parallel, delayed
import math

In [4]:
# --------------------------------------
# STEP 1: Import data
# --------------------------------------
datadir = '../data/'
jobposting_file = os.path.join(datadir, 'sample_job_postings_with_tasks-mapping.csv')

posting_df = pd.read_csv(jobposting_file)

In [5]:
posting_df.head()

,task,job_posting_index,embedding,cluster_kmeans,standardized_activity
0,identify and nurture leads to help expand the ...,0,"[0.007788926362991333, 0.03043454885482788, 0....",3,Lead generation and relationship management in...
1,reach out to potential clients via email and p...,0,"[-0.0873531699180603, 0.035773009061813354, 0....",192,Client Engagement and Outreach Activities
2,qualify leads,0,"[0.0034667470026761293, -0.014017626643180847,...",126,Lead Generation and Qualification
3,schedule meetings for account executives,0,"[0.0006416613468900323, -0.020684655755758286,...",71,Scheduling and setting up meetings for sales t...
4,analyze large datasets related to healthcare,1,"[0.05672462657094002, 0.04137267917394638, -0....",103,"Data collection, cleaning, and analysis, parti..."


In [6]:
# --------------------------------------
# STEP 2: Collapse to unique activity list
# --------------------------------------
activity_df = posting_df[["cluster_kmeans", "standardized_activity"]].drop_duplicates().reset_index(drop=True)

In [7]:
activity_df = activity_df.sort_values(by="cluster_kmeans").reset_index(drop=True)
activity_df.head()

,cluster_kmeans,standardized_activity
0,0,Legal Research and Case Strategy Development
1,1,Team Collaboration and Communication Specialist
2,2,Collaboration with cross-functional teams for ...
3,3,Lead generation and relationship management in...
4,4,Facilitating professional development and trai...


## Section 2: Use GPT to get AI exposure scores for each standardized task

In [9]:
# --------------------------------------
# STEP 1: Define GPT prompt function 
# --------------------------------------
import openai
import time

client = openai.OpenAI(api_key=(your API key here))

def classify_ai_impact_separately(activity_text):
    prompt = f"""
    You are an expert in artificial intelligence and labor economics. Your task is to classify the following job activity based on how current AI systems (e.g., GPT-4, automation tools, ML models) can either substitute for or complement human work.
    
    Provide **two numerical scores** for the task, each chosen from **0**, **0.5**, or **1**:
    
    ---
    
    1. **ai_substitutability**  
       - `1` if AI tools can **fully automate** or carry out **most core aspects** of the task independently and reliably.  
       - `0.5` if AI can help with **limited or peripheral parts** (e.g., summarizing, formatting, generating drafts), but humans still do the **main reasoning, oversight, or interaction**.  
       - `0` if AI offers **no meaningful ability** to perform or automate the task.
    
    2. **ai_complementarity**  
       - `1` if AI can **significantly support or enhance** performance of the task across **most dimensions**.  
       - `0.5` if AI helps with **some aspects**, but **cannot meaningfully enhance** the overall effectiveness of task completion.  
       - `0` if AI offers **little to no meaningful assistance** in doing this task better, faster, or more efficiently.
    
    ---
    
    **⚠️ Note**:  
    Do not assign `1` unless **AI meaningfully handles or enhances the majority** of the task. For tasks that involve physical presence, emotional intelligence, human coordination, or live teaching/facilitation, prefer a score of `0.5` or `0`.
    
    ---
    
    ### Examples
    
    1. Task: “Analyzing sales data using Excel”  
       → ai_substitutability: 1  
       → ai_complementarity: 1  
    
    2. Task: “Designing a marketing strategy for a new product”  
       → ai_substitutability: 0.5  
       → ai_complementarity: 1  
    
    3. Task: “Managing interpersonal conflict between coworkers”  
       → ai_substitutability: 0  
       → ai_complementarity: 0  
    
    4. Task: “Organizing and managing community events”  
       → ai_substitutability: 0  
       → ai_complementarity: 0.5  
    
    5. Task: “Compliance training and collaboration”  
       → ai_substitutability: 0.5  
       → ai_complementarity: 0.5  
    
    ---
    
    Now classify the following task:
    
    **Task**: "{activity_text}"
    
    Respond with:
    ai_substitutability: [0, 0.5, or 1]  
    ai_complementarity: [0, 0.5, or 1]
    """
   
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0,
        )
        content = response.choices[0].message.content.lower()

        # Just split on lines and extract numbers directly
        lines = content.splitlines()
        subst = float(lines[0].split(":")[1].strip())
        comp = float(lines[1].split(":")[1].strip())

        return pd.Series([subst, comp])

    except Exception as e:
        print(f"Error: {e}")
        return pd.Series([None, None])

In [10]:
# --------------------------------------
# STEP 2: Apply this to activity_df
# --------------------------------------
# Apply the classifier to each standardized activity
activity_df[["ai_substitutable", "ai_complementary"]] = activity_df["standardized_activity"].apply(classify_ai_impact_separately)

# Save result
activity_df.to_csv("../data/activity_ai_impact_flags.csv", index=False)

In [11]:
activity_df.head()

,cluster_kmeans,standardized_activity,ai_substitutable,ai_complementary
0,0,Legal Research and Case Strategy Development,0.5,1.0
1,1,Team Collaboration and Communication Specialist,0.0,0.5
2,2,Collaboration with cross-functional teams for ...,0.0,0.5
3,3,Lead generation and relationship management in...,0.5,1.0
4,4,Facilitating professional development and trai...,0.0,0.5


In [12]:
# --------------------------------------
# STEP 3: Check proportion of tasks that are AI-substitutable vs. AI-complementary
# --------------------------------------
# Count values for each column
comp_counts = activity_df['ai_complementary'].value_counts().sort_index()
subst_counts = activity_df['ai_substitutable'].value_counts().sort_index()

# Combine into a single DataFrame for neat display
summary_df = pd.DataFrame({
    'AI Complementarity': comp_counts,
    'AI Substitutability': subst_counts
}).fillna(0).astype(int)

# Display the summary
print(summary_df)

     AI Complementarity  AI Substitutability
0.0                   0                   31
0.5                  37                  154
1.0                 163                   15
